In [52]:
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
import pandas as pd


options = webdriver.ChromeOptions()

# options.add_argument('headless')

# set the window size
options.add_argument('window-size=1200x600')

# initialize the driver
driver = webdriver.Chrome(chrome_options=options)

# https://www.linkedin.com/uas/login
driver.get('https://www.linkedin.com/uas/login')

username = driver.find_element_by_id('session_key-login')
password = driver.find_element_by_id('session_password-login')

username.send_keys('ryan@growthgenius.com')
password.send_keys('Gr0wthP1ease1!')
# username.send_keys('srvansia@edu.uwaterloo.ca')
# password.send_keys('!eat@2468')
driver.find_element_by_name("signin").click()
df = pd.read_csv("Outbound.csv")


In [47]:
#get job

def get_job(path, destination, present):
    attempts = 0;
    while(attempts<2):
        try:
            if present:
                present_info = driver.find_element_by_xpath(path)
                datastore[destination] = present_info.text
#                 print (datastore[destination])

            else:
                past_info = driver.find_element_by_xpath(path)
                datastore[destination] = past_info.text
#                 print (datastore[destination])
           
            break
        except (StaleElementReferenceException, NoSuchElementException) as e:
            if e == StaleElementReferenceException:
                attempts += 1
            else:
                attempts = 4

In [48]:
def get(path, destination, personal):
    attempts = 0;
    while(attempts<2):
        try:
            if personal:
                personal_info = driver.find_element_by_xpath(path)
                datastore[destination] = personal_info.text
#                 print (datastore[destination])
            else:
                pass
            break
        except (StaleElementReferenceException, NoSuchElementException) as e:
            if e == StaleElementReferenceException:
                attempts += 1
            else:
                attempts = 4

In [49]:
#scroll to the end of page
def scroll_to_end_of_page():
    scrolled_amount = 0
    height = driver.execute_script("return document.body.scrollHeight")
    while scrolled_amount < height:
        scrolled_amount+=100
        driver.execute_script("window.scrollTo(0, %s);" % scrolled_amount)
        time.sleep(0.2)
        height = driver.execute_script("return document.body.scrollHeight")

In [50]:
datastore = {
                 'Name': '',
                 'Connection': '',
                 'Headline': '', 
                 'Location': '', 
                 'Description': '', 
            
                 'Present Job Title': '',
                 'Company': '',
                 'Date': '',
                 'Duration': '',
                 'Loc': '',
                 'Des': '',
       
                 'Past Job Title': '',
                 'Past Company': '',
                 'Past Date': '',
                 'Past Dur': '',
                 'Past Loc': '',
                 'Past Description': ''
            
        }


In [51]:

i = 0
t = 5
for x in df["Person Linkedin Url"].tolist():
    if i < t:
        driver.get(x)
        if driver.current_url != "https://www.linkedin.com/in/unavailable/":
        
            scroll_to_end_of_page()
            time.sleep(10)
            i += 1

    # Personal Information
            name = "//h1[@class='pv-top-card-section__name Sans-26px-black-85%']"
            get(name, 'Name', True) 
            first_name = datastore['Name'].split(" ")[0]
            connection = "//span[@class = 'dist-value']"
            get(connection, 'Connection', True)
            headline = "//h2[@class='pv-top-card-section__headline mt1 Sans-19px-black-85%']"      
            get(headline, 'Headline', True)
            location = "//h3[@class='pv-top-card-section__location Sans-17px-black-55%-dense mt1 inline-block']"
            get(location, 'Location', True)
            description = "//div[@class='pv-top-card-section__summary pv-top-card-section__summary--with-content mt4 ember-view']/p"
            get(description, 'Description', True)

    # Present Company Information
            pres_title = "//section[@id ='experience-section']//li[1]//h3"
            get_job(pres_title, 'Present Job Title', True)            
            pres_company = "//section[@id ='experience-section']//li[1]//h4[1]/span[2]"
            get_job(pres_company, 'Company', True)
            pres_date = "//h4[@class='pv-entity__date-range inline-block Sans-15px-black-70%']"
            get_job(pres_date, 'Date', True)            
            pres_dur = "//span[@class='pv-entity__bullet-item']"
            get_job(pres_dur, 'Duration', True)
            pres_loc = "//h4[@class='pv-entity__location Sans-15px-black-70% block']"
            get_job(pres_loc, 'Loc', True)
            pres_des = "//p[@class='pv-entity__description Sans-15px-black-70% mt4']"
            get_job(pres_des, 'Des', True)

    # Past company info            
            past_title = "//section[@class ='pv-profile-section experience-section ember-view']/ul/li[2]/a/div/h3"
            get_job(past_title, 'Past Company Title', False)
            past_comp = "//section[@class ='pv-profile-section experience-section ember-view']/ul/li[2]/a/div/h4[@class='Sans-17px-black-85%']/span[2]"
            get_job(past_comp, 'Past Company', False)
            past_date = "//section[@class ='pv-profile-section experience-section ember-view']/ul/li[2]/a/div/h4[2]/span[2]"
            get_job(past_date, 'Past Date', False)
            past_dur = "//section[@class ='pv-profile-section experience-section ember-view']/ul/li[2]/a/div/h4[3]/span[2]"
            get_job(past_dur, 'Past Dur', False)
            past_loc = "//section[@class ='pv-profile-section experience-section ember-view']/ul/li[2]/a/div/h4[4]/span[2]"
            get_job(past_loc, 'Past Loc', False)
            past_des = "//section[@class ='pv-profile-section experience-section ember-view']/ul/li[2]/div[@class='pv-entity__extra-details ember-view']/p"
            get_job(past_des, 'Past Description', False)

    # Scroll to the top
            driver.execute_script("window.scrollTo(0, 0);")


    # Sending a request            
            try:
                connect = driver.find_element_by_css_selector('.pv-s-profile-actions.pv-s-profile-actions--connect.button-primary-large.mr2.mt2').click()
                msg_but = driver.find_element_by_css_selector('.button-secondary-large.mr1').click()

            except NoSuchElementException:
                connect = driver.find_element_by_css_selector('.pv-s-profile-actions__overflow-toggle.pv-top-card-section__inline-overflow-button.button-secondary-large-muted.mt2').click()
                msg_but = driver.find_element_by_css_selector('.pv-s-profile-actions.pv-s-profile-actions--connect.pv-s-profile-actions__overflow-button.full-width.text-align-left').click()
                driver.find_element_by_css_selector('.button-secondary-large.mr1').click()
     # Sending a Message        
            note = driver.find_element_by_id("custom-message")
            note.send_keys("Hey " + first_name + ", your profile caught my eye. Would love to learn about your story with "+datastore['Company']+" and your experiences. Let's chat upon connecting!")
            driver.find_element_by_css_selector('.button-primary-large.ml1').click()
        
            
    else:
        break
    
    